In [1]:
# Tell python where to look for modules. 
# Depending on how your jupyter handles working directories, this may not be needed.
import sys
sys.path.append('../../hourly-egrid/')

In [2]:
# import the necessary packages
%reload_ext autoreload
%autoreload 2

# import packages
import os
import requests
import tarfile
import sqlalchemy as sa
from pathlib import Path
import pandas as pd
import plotly.express as px
import numpy as np

import src.data_cleaning as data_cleaning
import src.gross_to_net_generation as gross_to_net_generation
import src.load_data as load_data
import src.distribute_eia923 as distribute_eia923

C:\Users\Greg\miniconda3\envs\hourly_egrid\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


# Specify the year for analysis

In [3]:
year = 2020

# 1. Download data

 - Downloads the pre-cleaned PUDL versions of EIA-923, EIA-860, and EPA CEMS data  
 - Downloads EPA eGRID data  
 - Downloads EIA-930 data  
 - Downloads the EPA Power Sector Data Crosswalk

TODO
- [x] The code for downloading the files could probably be made into functions
- [ ] Investigate other packages besides `requests` that would download these files faster

In [4]:
############### PUDL Database ######################

load_data.download_pudl_data(zenodo_url = 'https://zenodo.org/record/6349861/files/pudl-v0.6.0-2022-03-12.tgz')

################# eGRID data #########################

# the 2019 and 2020 data appear to be hosted on different urls
egrid_files_to_download = ['https://www.epa.gov/sites/default/files/2021-02/egrid2019_data.xlsx', 
                           'https://www.epa.gov/system/files/documents/2022-01/egrid2020_data.xlsx']

load_data.download_egrid_files(egrid_files_to_download)

############# EIA-930 data #####################

load_data.download_eia930_data(years_to_download=[year])

########## Power Sector Data Crosswalk #############
# NOTE: Check for new releases at https://github.com/USEPA/camd-eia-crosswalk

load_data.download_epa_psdc(psdc_url='https://github.com/USEPA/camd-eia-crosswalk/releases/download/v0.2.1/epa_eia_crosswalk.csv')


PUDL data already downloaded
egrid2019_data.xlsx already downloaded
egrid2020_data.xlsx already downloaded
2020_Jan_Jun data already downloaded
2020_Jul_Dec data already downloaded
epa_eia_crosswalk.csv already downloaded


# Load emissions data reported to CEMS
There are three broad categories of plants based on their CAMD reporting status:
1. Units that report to CAMD year-round (for these plants, emissions data is used directly from CEMS)
2. Units that only report to CAMD during the ozone season (May-Sept) (for these units, non-ozone season data is taken from EIA 923)
3. Units that do not report to CAMD (generally fossil units < 25MW and non-fossil generators)

There are also certain plants that report to CAMD but do not procuce electricty for the grid, and need to be removed from the CEMS data:
- Non grid connected plants
- Steam-only plants


In [ ]:
# NOTE: all of the functions in this section could be run by calling clean_cems()
#cems = data_cleaning.clean_cems(year)

In [5]:
# load the CEMS data
cems = load_data.load_cems_data(year)

# remove non-grid connected plants
cems = data_cleaning.remove_non_grid_connected_plants(cems)

# remove plants that only report steam generation and no electrical generation
cems = data_cleaning.remove_heating_only_plants(cems)

# add a report date
cems = data_cleaning.add_report_date(cems)

# identify cems reporting status
cems = data_cleaning.determine_cems_reporting_status(cems)

# TODO: identify and remove any hourly values that appear to be outliers


# fill in missing hourly emissions data using the fuel type and heat input
cems = data_cleaning.fill_cems_missing_co2(cems, year)

Removing 10 plants that are not grid-connected
Removing 104 plants that only produce heat and no power


In [6]:
# identify any remaining missing values
# TODO: Try to identify fuel types
# NOTE: plant 880109 appears to be a paper mill in Ohio, so should maybe be added to non-grid connected

units_with_no_fuel_type = list(cems[cems['co2_mass_tons'].isnull()]['cems_id'].unique())
print(f"Unable to find fuel types for the following plants_units: {units_with_no_fuel_type}")

cems[cems['co2_mass_tons'].isnull()]

Unable to find fuel types for the following plants_units: ['1004_CTG1', '880109_B001']


,plant_id_eia,unitid,cems_id,operating_datetime_utc,operating_time_hours,gross_load_mw,gross_generation_mwh,steam_load_1000_lbs,heat_content_mmbtu,co2_mass_tons,co2_mass_measurement_code,plant_id_epa,unit_id_epa,report_date,cems_reporting_category,energy_source_code
10905359,1004,CTG1,1004_CTG1,2020-04-10 04:00:00+00:00,0.03,0.0,0.0,NaN,0.000900,NaN,Measured,1004,90673,2020-04-01,full_year,SGC
11497174,1004,CTG1,1004_CTG1,2020-11-02 03:00:00+00:00,0.03,0.0,0.0,NaN,0.000900,NaN,Measured,1004,90673,2020-11-01,full_year,SGC
23945496,880109,B001,880109_B001,2020-05-01 05:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,<NA>,880109,91300,2020-05-01,partial_year,NaN
23945497,880109,B001,880109_B001,2020-05-01 06:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,<NA>,880109,91300,2020-05-01,partial_year,NaN
23945498,880109,B001,880109_B001,2020-05-01 07:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,<NA>,880109,91300,2020-05-01,partial_year,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24290251,880109,B001,880109_B001,2020-10-01 00:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,<NA>,880109,91300,2020-09-01,partial_year,NaN
24290252,880109,B001,880109_B001,2020-10-01 01:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,<NA>,880109,91300,2020-09-01,partial_year,NaN
24290253,880109,B001,880109_B001,2020-10-01 02:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,<NA>,880109,91300,2020-09-01,partial_year,NaN
24290254,880109,B001,880109_B001,2020-10-01 03:00:00+00:00,1.00,0.0,0.0,NaN,332.299988,NaN,<NA>,880109,91300,2020-09-01,partial_year,NaN


In [7]:
# For now, lets drop these from the data
cems = cems[~cems['cems_id'].isin(units_with_no_fuel_type)]

In [8]:
# remove any observations from cems where zero operation is reported for an entire month
# although this data could be considered to be accurately reported, let's remove it so that we can double check against the eia data
# TODO: check if any of these observations are from geothermal generators
cems = data_cleaning.remove_cems_with_zero_monthly_emissions(cems)

removing 6990312 observations from cems


In [9]:
# add information about the balancing authority 
#cems = cems.drop(columns=['ba_code','state'])
cems = data_cleaning.assign_ba_code_to_plant(cems, year)

## Explore outlier detection
We need to come up with a method that filters out observations that are significantly higher than normal operation. The challenge is that some plants only operate a handful of hours each year, so their operation looks spikey, which would be identified as an outlier using typical detection methods. 

In [ ]:
max = cems.replace(0, np.NaN).groupby(['cems_id'])['heat_content_mmbtu'].max()
mean = cems.replace(0, np.NaN).groupby(['cems_id'])['heat_content_mmbtu'].mean()
stdev = cems.replace(0, np.NaN).groupby(['cems_id'])['heat_content_mmbtu'].std()

max[max > mean + (3 * stdev)]

In [ ]:
px.line(cems[cems['cems_id'] == '10298_CG803'], x='operating_datetime_utc', y='heat_content_mmbtu')

In [ ]:
px.line(cems[cems['cems_id'] == '1012_2'], x='operating_datetime_utc', y='heat_content_mmbtu')

# 2. Get monthly data for all plants/units where data is missing from CEMS
We have now identified all plants that report the full year to CEMS, and all plants that report a partial year. We will now use the EIA-923 data to fill in the missing pieces.

1. Load EIA-923 data, and standardize heat input and generation data across the tables
2. Identify all plants/months for which we do not have CEMS data

We need to be able to match the EIA data to the CEMS data based on units so we know which data will be used to fill the missing data


In [74]:
# Distribute net generation and heat input data reported by the three different EIA-923 tables
# NOTE: this code was copied and modified from `pudl.analysis.allocate_net_gen`
# NOTE: this code allocates net generation based on the proportion of net generation reported, rather than by nameplate capacity (which eGRID does)
# NOTE: the code was modified to perform the allocation on a monthly basis, rather than an annual basis

# HIGH PRIORITIES
# DONE: remove non-grid connected plants from this dataframe
# DONE: Denormalize data by balancing authority/state. BA assignment from EIA-860
# TODO: look into whether net_gen from generation table should be preserved if available
# TODO: figure out what happens when each table has different values for net gen (allocate difference, or take precedence)
# DONE: add nuclear generators to this list

# FUEL ASSIGNMENT
# DONE: calculate total emissions from gf based on fuel and heat input and distribute in addition to net generation and fuel consumed
# TODO: when aggregating back to generator records, keep the fuel type that accounted for most heat input
# TODO: denormalize data by fuel type both primary fuel type by generator, and primary fuel by plant (assuming that's how reported to ISOs)
# primary fuel type is currently assigned based on the annual primary fuel type. This should be changed to assign base on monthly fuel type

# NEXT PRIORITIES
# TODO: allocate heat input data from boiler_fuel_eia923() See: https://github.com/catalyst-cooperative/pudl/pull/1096
# TODO: Also distribute heat input for electricity consumption

# LOWER PRIORITIES
# TODO: fix allocation of net generation when reported net generation is negative?
# TODO: investigate generators for which frac column is not adding to 1.0

gen_fuel_allocated = distribute_eia923.allocate_gen_fuel_by_gen(year=year)

# flag any generator-months for which we already have cems data
gen_fuel_allocated = data_cleaning.identify_emissions_data_source(cems, gen_fuel_allocated)

# create a separate dataframe containing only the generators for which we do not have CEMS data
monthly_eia_data_to_distribute = gen_fuel_allocated[(gen_fuel_allocated['data_source'] == 'eia_only') & ~(gen_fuel_allocated['fuel_consumed_mmbtu'].isna())]

gen_fuel_allocated.sample(10)

Removing 0 plants that are not grid-connected
Removing 0 plants that are not grid-connected
Removing 1 plants that are not grid-connected


c:\Users\Greg\GitHub\hourly-egrid\notebooks\../../hourly-egrid\src\distribute_eia923.py:894: UserWarning: Ooopsies. You got 249 records where the 'frac' column isn't adding up to 1 for each 'IDX_PM_FUEL' group. Check 'calc_allocation_fraction()'
        plant_id_eia prime_mover_code energy_source_code report_date  frac  \
3413             377               CA                 NG  2020-04-01   2.0   
3414             377               CT                 NG  2020-04-01   2.0   
3441             377               CA                 NG  2020-11-01   2.0   
3442             377               CT                 NG  2020-11-01   2.0   
3445             377               CA                 NG  2020-12-01   2.0   
...              ...              ...                ...         ...   ...   
104977         58207               CA                 NG  2020-03-01   4.0   
104981         58207               CA                 NG  2020-04-01   4.0   
104989         58207               CA               

,plant_id_eia,generator_id,report_date,net_generation_mwh,fuel_consumed_mmbtu,co2_mass_tons,ba_code,state,energy_source_code_1,data_source
128761,52120,G-65,2020-04-01,32090.510000,641296.000000,37477.338240,ERCO,TX,NG,eia_only
209220,58436,ENG1,2020-05-01,1268.230500,12780.500000,811.561750,CISO,CA,LFG,eia_only
3987,153,8,2020-02-01,37410.731707,328204.367378,0.000000,WALC,AZ,WAT,eia_only
63930,3457,2,2020-12-01,85577.000000,966190.235561,56464.157366,MISO,TX,NG,cems
24927,1248,GT4,2020-06-01,3.696148,75.194346,6.140370,SWPP,KS,DFO,eia_only
255779,61795,12649,2020-03-01,NaN,NaN,0.000000,CISO,CA,MWH,eia_only
217536,58966,T 1-6,2020-05-01,2268.110000,19898.000000,0.000000,NWMT,MT,WND,eia_only
160060,55810,S4,2020-09-01,1016.872333,12556.666667,733.811600,CISO,CA,NG,cems
128046,52076,UNIT2,2020-09-01,4068.065000,50100.500000,2927.873220,CISO,CA,NG,eia_only
104384,10017,GEN9,2020-11-01,4250.000000,138755.313401,12639.690140,PJM,VA,BLQ,eia_only


In [69]:
# for which plants are we still missing co2 data?
gen_fuel_allocated[(gen_fuel_allocated['co2_mass_tons'].isna()) & (gen_fuel_allocated['fuel_consumed_mmbtu'] > 0)]

,plant_id_eia,generator_id,report_date,net_generation_mwh,fuel_consumed_mmbtu,co2_mass_tons,ba_code,state,energy_source_code_1,data_source
123042,50626,GEN1,2020-01-01,742.746,381046.0,NaN,MISO,LA,OTH,eia_only
123043,50626,GEN1,2020-02-01,700.137,359168.0,NaN,MISO,LA,OTH,eia_only
123044,50626,GEN1,2020-03-01,654.929,335974.0,NaN,MISO,LA,OTH,eia_only
123045,50626,GEN1,2020-04-01,628.143,322234.0,NaN,MISO,LA,OTH,eia_only
123046,50626,GEN1,2020-05-01,562.301,288461.0,NaN,MISO,LA,OTH,eia_only
123047,50626,GEN1,2020-06-01,626.327,321326.0,NaN,MISO,LA,OTH,eia_only
123048,50626,GEN1,2020-07-01,702.505,360388.0,NaN,MISO,LA,OTH,eia_only
123049,50626,GEN1,2020-08-01,836.592,429186.0,NaN,MISO,LA,OTH,eia_only
123050,50626,GEN1,2020-09-01,730.017,374505.0,NaN,MISO,LA,OTH,eia_only
123051,50626,GEN1,2020-10-01,829.739,425649.0,NaN,MISO,LA,OTH,eia_only


In [70]:
# investigate single plants
gen_fuel_allocated[gen_fuel_allocated['plant_id_eia'] == 50150]

,plant_id_eia,generator_id,report_date,net_generation_mwh,fuel_consumed_mmbtu,co2_mass_tons,ba_code,state,energy_source_code_1,data_source
116062,50150,GE10,2020-01-01,6720.160000,70743.048847,4134.223775,ERCO,TX,NG,eia_only
116063,50150,GE10,2020-02-01,6567.870000,46835.963368,2737.093699,ERCO,TX,NG,eia_only
116064,50150,GE10,2020-03-01,8431.240000,58009.611327,3390.081686,ERCO,TX,NG,eia_only
116065,50150,GE10,2020-04-01,909.860000,11653.710913,681.042866,ERCO,TX,NG,eia_only
116066,50150,GE10,2020-05-01,4022.590000,30746.534815,1796.827495,ERCO,TX,NG,eia_only
...,...,...,...,...,...,...,...,...,...,...
116153,50150,IGT,2020-08-01,6998.671282,88392.102564,5165.634474,ERCO,TX,NG,eia_only
116154,50150,IGT,2020-09-01,5775.168821,72939.384615,4262.577637,ERCO,TX,NG,eia_only
116155,50150,IGT,2020-10-01,4994.167795,63075.487179,3686.131471,ERCO,TX,NG,eia_only
116156,50150,IGT,2020-11-01,3911.813333,49405.538462,2887.259668,ERCO,TX,NG,eia_only


In [ ]:
# investigate plants that don't have fuel codes
plants = load_data.load_pudl_table("plants_entity_eia")
gf = load_data.load_pudl_table("generation_fuel_eia923", year=year)
#plants[plants['plant_id_eia'] == 50626]
#gf[gf['plant_id_eia'] == 50626
# look at plants that are refineries
#gf[gf['plant_id_eia'].isin(list(plants.fillna('').loc[plants.fillna('')['plant_name_eia'].str.contains('refin', case=False)]['plant_id_eia']))]

In [12]:
# what percent of generators are in CEMS vs not
gen_fuel_allocated.groupby('data_source').sum() / gen_fuel_allocated.groupby('data_source').sum().sum(axis=0)

,plant_id_eia,net_generation_mwh,fuel_consumed_mmbtu,co2_mass_tons
data_source,,,,
cems,0.108607,0.546135,0.500088,0.797197
eia_only,0.891393,0.453865,0.499912,0.202803


## Calculate CEMS net generation
Now that we have accurate net generation data from EIA, we can use this to calculate a net generation ratio to convert the CEMS gross generation to hourly net generation

For now, we will calculate the allocation at the plant level. However, in the future, we may want to calculate for each EPA unit, which will require developing a scheme for allocating each EPA unit to EIA generator

In [13]:
cems = data_cleaning.convert_gross_to_net_generation(cems, gen_fuel_allocated)

C:\Users\Greg\miniconda3\envs\hourly_egrid\lib\site-packages\statsmodels\regression\linear_model.py:1673: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\Greg\miniconda3\envs\hourly_egrid\lib\site-packages\numpy\linalg\linalg.py:2158: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
C:\Users\Greg\miniconda3\envs\hourly_egrid\lib\site-packages\statsmodels\regression\linear_model.py:1673: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\Greg\miniconda3\envs\hourly_egrid\lib\site-packages\numpy\linalg\linalg.py:2158: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
C:\Users\Greg\miniconda3\envs\hourly_egrid\lib\site-packages\statsmodels\regression\linear_model.py:1673: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid

In [14]:
# what percent of gross generation was allocated using each method?
cems.groupby('net_gen_method', dropna=False).sum()['gross_generation_mwh'] / cems['gross_generation_mwh'].sum()

net_gen_method
annual_regression    0.032328
monthly_ratio        0.967672
Name: gross_generation_mwh, dtype: float32

# Adjust emissions
We next need to make certain adjustments to the data:
 - [ ] Calculate emissions for Geothermal plants
 - [ ] Adjust heat input/emissions from CHP plants by proportion used for electric generation
 - EPA adjusts biomass emissions, but not sure if we want to do that. Need to look into it more

# Output CEMS data

In [ ]:
# output the cems data to csv for others to use (update the date)
cems_for_export = cems.loc[cems['co2_mass_tons'] > 0, ['plant_id_eia', 'unitid', 'operating_datetime_utc',
       'gross_generation_mwh','net_generation_mwh','steam_load_1000_lbs', 'heat_content_mmbtu', 'co2_mass_tons',
       'report_date', 'cems_reporting_category', 'energy_source_code',
       'ba_code', 'state']]

date = '20220415'
cems_for_export.to_csv(f'../data/output/cems_{year}_cleaned_{date}.csv')

# Compare results to eGRID totals

Before we allocate the data to the hourly level, we should double check that the total annual emissions / generation values match the "official" data published in eGRID at the annual level.

In [75]:
# Aggregate total calculated values
###################################

# combine cems and eia data
cems_plant_annual = cems.groupby(['ba_code','state','plant_id_eia'], dropna=False).sum()[['net_generation_mwh','heat_content_mmbtu','co2_mass_tons']].rename(columns={'heat_content_mmbtu':'heat_input_mmbtu'}).reset_index()
eia_plant_annual = monthly_eia_data_to_distribute.groupby(['ba_code','state','plant_id_eia'], dropna=False).sum()[['net_generation_mwh','fuel_consumed_mmbtu','co2_mass_tons']].rename(columns={'fuel_consumed_mmbtu':'heat_input_mmbtu'}).reset_index()

plant_annual_total = pd.concat([cems_plant_annual,eia_plant_annual], axis=0)

# group any plants that have records from both datasets
plant_annual_total = plant_annual_total.groupby(['ba_code','state','plant_id_eia'], dropna=False).sum().reset_index()

# Load the eGRID plant table
############################

# load plant level data from egrid
egrid_plant = pd.read_excel(f'../data/egrid/egrid{year}_data.xlsx', 
                            sheet_name=f'PLNT{str(year)[-2:]}', 
                            header=1, 
                            usecols=['BACODE','PSTATABB', 'PLPRMFL','ORISPL', 'PNAME','PLGENATN', 'PLGENATR', 'PLHTIANT','UNCO2','PLCO2AN'])
# calculate total net generation
egrid_plant['net_generation_mwh'] = egrid_plant['PLGENATN'] + egrid_plant['PLGENATR']
egrid_plant = egrid_plant.drop(columns=['PLGENATN', 'PLGENATR'])
# rename the columns
egrid_plant = egrid_plant.rename(columns={'BACODE':'ba_code',
                                          'PSTATABB':'state',
                                          'PLPRMFL':'energy_source_code',
                                          'ORISPL':'plant_id_eia',
                                          'PNAME':'plant_name',
                                          'PLHTIANT':'heat_input_mmbtu',
                                          'UNCO2':'co2_mass_tons_unadjusted',
                                          'PLCO2AN':'co2_mass_tons'})

# if egrid has a missing value for co2 for a clean plant, replace with zero
clean_fuels = ['SUN','MWH','WND', 'WAT','WH','PUR','NUC']
egrid_plant.loc[egrid_plant['energy_source_code'].isin(clean_fuels), 'co2_mass_tons'] = egrid_plant.loc[egrid_plant['energy_source_code'].isin(clean_fuels), 'co2_mass_tons'].fillna(0)
egrid_plant.loc[egrid_plant['energy_source_code'].isin(clean_fuels), 'co2_mass_tons_unadjusted'] = egrid_plant.loc[egrid_plant['energy_source_code'].isin(clean_fuels), 'co2_mass_tons_unadjusted'].fillna(0)

# reorder the columns                                         
egrid_plant = egrid_plant[['ba_code', 'state', 'plant_id_eia', 'plant_name', 'net_generation_mwh', 'heat_input_mmbtu', 'co2_mass_tons', 'co2_mass_tons_unadjusted']]

# egrid did not renumber its plant ids to match the EIA ids, so we need to crosswalk these
egrid_crosswalk = pd.read_csv('../data/egrid/egrid_static_tables/table_C5_crosswalk_of_EIA_ID_to_EPA_ID.csv')
epaid_to_eiaid = dict(zip(list(egrid_crosswalk['EPA/CAMD ORISPL ID']), list(egrid_crosswalk['EIA ORISPL ID'])))
print(f" Updatating {len(egrid_plant[egrid_plant['plant_id_eia'].isin(list(egrid_crosswalk['EPA/CAMD ORISPL ID']))])} plant codes from eGRID")

egrid_plant['plant_id_eia'].update(egrid_plant['plant_id_eia'].map(epaid_to_eiaid))

 Updatating 21 plant codes from eGRID


## Identify plants that are missing from each dataset

In [76]:
# identify any plants that are in egrid but not our totals, and any plants that are in our totals, but not egrid
plants_not_in_total = list(set(egrid_plant['plant_id_eia'].unique()) - set(plant_annual_total['plant_id_eia'].unique()))
plants_not_in_egrid = list(set(plant_annual_total['plant_id_eia'].unique()) - set(egrid_plant['plant_id_eia'].unique()))

# Which plants are included in eGRID but are missing from our calculations?
missing_from_calc = egrid_plant[egrid_plant['plant_id_eia'].isin(plants_not_in_total)]
missing_from_calc

,ba_code,state,plant_id_eia,plant_name,net_generation_mwh,heat_input_mmbtu,co2_mass_tons,co2_mass_tons_unadjusted
0,NaN,AK,60814,7-Mile Ridge Wind Project,NaN,NaN,0.0,0.0
1,NaN,AK,54452,Agrium Kenai Nitrogen Operations,NaN,NaN,NaN,NaN
18,NaN,AK,93,Blue Lake Hydro,NaN,NaN,0.0,0.0
49,NaN,AK,313,Green Lake,NaN,NaN,0.0,0.0
53,NaN,AK,59037,Hiilangaay Hydro,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
12642,WAUW,WY,674,Pilot Butte,NaN,NaN,0.0,0.0
12644,WACM,WY,64847,Rail Tie Wind,NaN,NaN,0.0,0.0
12659,PACE,WY,62516,TB Flats,NaN,NaN,0.0,0.0
12662,PACE,WY,63972,Two Rivers Wind Facility,NaN,NaN,0.0,0.0


In [77]:
# of these plants that are missing from our data, how many of them have non-zero data in eGRID?
plants_with_no_data_in_egrid = list(missing_from_calc[missing_from_calc[['net_generation_mwh','heat_input_mmbtu','co2_mass_tons','co2_mass_tons_unadjusted']].sum(axis=1) == 0]['plant_id_eia'])
missing_from_calc = missing_from_calc[~missing_from_calc['plant_id_eia'].isin(plants_with_no_data_in_egrid)]
# How many are not located in Puerto Rico?
missing_from_calc = missing_from_calc[(missing_from_calc['state'] != 'PR')]
missing_from_calc.to_csv('../data/temp/plants_missing_from_calcs.csv', index=False)
# see if any of these plants are retired
generators_eia860 = load_data.load_pudl_table('generators_eia860', year=year)
missing_from_calc.merge(generators_eia860.groupby('plant_id_eia')['retirement_date'].unique().reset_index(), how='left', on='plant_id_eia')

,ba_code,state,plant_id_eia,plant_name,net_generation_mwh,heat_input_mmbtu,co2_mass_tons,co2_mass_tons_unadjusted,retirement_date
0,SOCO,AL,50359,Sloss Industries Corp,0.0,0.0,0.000,96648.513,[None]
1,CISO,CA,328,Borel,-204.0,NaN,0.000,0.000,[2017-09-01]
2,CISO,CA,302,Cabrillo Power I Encina Power Station,388042.0,4474181.0,265896.801,265896.801,NaN
3,MISO,IA,7922,Brooklyn City North Plant,42.0,NaN,NaN,NaN,[2011-09-01]
4,MISO,IL,7854,Avenue A Generator Sets,162.0,1646.0,134.418,134.418,[None]
5,SWPP,KS,64074,Cimarron Bend III,52458.0,NaN,0.000,0.000,[None]
6,MISO,LA,60610,St. Mary Clean Energy Center,179664.0,NaN,0.000,0.000,[None]
7,ISNE,ME,10491,Mead Rumford Cogen,0.0,0.0,0.000,58381.352,[None]
8,MISO,MN,63304,"Allium Community Solar Garden, LLC",62.0,NaN,0.000,0.000,[None]
9,MISO,MN,1979,Hibbing,0.0,0.0,0.000,54323.000,[None]


In [78]:
# Which plants are in our calculations, but are missing from eGRID?
plant_names = load_data.load_pudl_table('plants_entity_eia')[['plant_id_eia','plant_name_eia','sector_name_eia']]
missing_from_egrid = plant_annual_total[plant_annual_total['plant_id_eia'].isin(plants_not_in_egrid)].merge(plant_names, how='left', on='plant_id_eia')

missing_from_egrid

,ba_code,state,plant_id_eia,net_generation_mwh,heat_input_mmbtu,co2_mass_tons,plant_name_eia,sector_name_eia
0,CISO,CA,55508,1.901500e+04,1.932511e+05,1.133705e+04,CalPeak Power Panoche No. 2,IPP Non-CHP
1,CISO,CA,59002,3.880420e+05,4.274786e+06,2.540470e+05,Carlsbad Energy Center,IPP Non-CHP
2,CPLE,NC,7538,1.753980e+05,2.448530e+06,1.467698e+05,Wayne County,Electric Utility
3,ERCO,TX,3612,8.574230e+05,9.627642e+06,5.721984e+05,V H Braunig,Electric Utility
4,ISNE,CT,562,2.948827e+05,3.376642e+06,2.069565e+05,Middletown,IPP Non-CHP
5,MISO,IN,10397,1.947440e+05,8.026617e+06,1.524149e+06,Indiana Harbor,Industrial CHP
6,MISO,WI,4076,1.840100e+04,3.025524e+05,1.791048e+04,West Marinette,Electric Utility
7,NYIS,NY,55375,2.578380e+06,1.850429e+07,1.099723e+06,Astoria Energy,IPP Non-CHP
8,SRP,AZ,55306,2.486618e+06,1.847669e+07,1.098044e+06,Panda Union Power Partners LP,None
9,SRP,AZ,55481,3.916020e+06,2.796750e+07,1.662070e+06,Mesquite Generating Station Block 2,IPP Non-CHP


In [79]:
# for some reason, egrid removes certain plants that are found in the ORIS crosswalk, and then never adds them back in. 
# It seems that these plants should be kept, and this might be a mistake in eGRID
plants_removed_from_egrid = pd.read_csv('../data/egrid/egrid_static_tables/table_C5_crosswalk_of_EIA_ID_to_EPA_ID.csv')
plants_removed_from_egrid = plants_removed_from_egrid.rename(columns={'EIA ORISPL ID':'plant_id_eia','EIA Plant Name':'plant_name_eia','EPA/CAMD ORISPL ID':'plant_id_epa','EPA/CAMD Plant Name':'plant_name_epa'})

# which plants are missing from egrid that weren't intentionally removed?
missing_from_egrid[~missing_from_egrid['plant_id_eia'].isin(list(plants_removed_from_egrid['plant_id_epa'].unique()))]

,ba_code,state,plant_id_eia,net_generation_mwh,heat_input_mmbtu,co2_mass_tons,plant_name_eia,sector_name_eia
1,CISO,CA,59002,3.880420e+05,4.274786e+06,2.540470e+05,Carlsbad Energy Center,IPP Non-CHP
2,CPLE,NC,7538,1.753980e+05,2.448530e+06,1.467698e+05,Wayne County,Electric Utility
5,MISO,IN,10397,1.947440e+05,8.026617e+06,1.524149e+06,Indiana Harbor,Industrial CHP
10,SRP,AZ,59338,3.165264e+06,4.632723e+07,2.753168e+06,PB1 - Gila River,IPP Non-CHP
12,NaN,NaN,55248,7.002651e+05,8.503199e+06,5.016617e+05,Tait,IPP Non-CHP
13,NaN,NaN,880075,0.000000e+00,1.026400e+06,5.998282e+04,BP Amoco Chemical Company,None
14,NaN,NaN,880079,3.290697e+05,4.952602e+06,2.894301e+05,Tate & Lyle-Loudon,None


## Identify plants for which we are missing a BA assignment
(of the plants not already missing from our calculated totals)

In [80]:
ba_code_match = egrid_plant.set_index('plant_id_eia')[['plant_name','ba_code']].merge(plant_annual_total.set_index('plant_id_eia')[['ba_code']], how='inner', left_index=True, right_index=True, suffixes=("_egrid",'_calc'))

# plants with missing ba code
ba_code_match[(ba_code_match['ba_code_calc'].isna()) & ~(ba_code_match['ba_code_egrid'].isna())]

,plant_name,ba_code_egrid,ba_code_calc
plant_id_eia,,,


## Identify plants for which we have incorrectly assigned the BA code

In [81]:
# plants with incorrect ba code
ba_code_match[(ba_code_match['ba_code_calc'] != ba_code_match['ba_code_egrid']) & ~(ba_code_match['ba_code_calc'].isna())]

,plant_name,ba_code_egrid,ba_code_calc
plant_id_eia,,,


## Identify where our calculated totals do not match eGRID's totals

In [82]:
# standardize column names and index so that the two dfs can be divided
calculated_to_compare = plant_annual_total.set_index('plant_id_eia').drop(columns=['ba_code','state'])
calculated_to_compare['co2_mass_tons_unadjusted'] = calculated_to_compare['co2_mass_tons']

# drop the plants that have no data in eGRID
egrid_plant = egrid_plant[~egrid_plant['plant_id_eia'].isin(plants_with_no_data_in_egrid)]

egrid_to_compare = egrid_plant.set_index(['plant_id_eia']).drop(columns=['ba_code','state','plant_name'])

# divide calculated value by egrid value
compared = calculated_to_compare.div(egrid_to_compare).merge(egrid_plant[['plant_id_eia','plant_name','ba_code', 'state']], how='left', left_index=True, right_on='plant_id_eia').set_index('plant_id_eia')
compared['plant_name'] = compared['plant_name'].fillna('unknown')

# create a dataframe that merges the two sources of data together
compared_merged = calculated_to_compare.merge(egrid_to_compare, how='outer', on='plant_id_eia', suffixes=('_calc','_egrid'))

# for each column, change missing values to zero if both values are zero (only nan b/c divide by zero)
for col in ['net_generation_mwh','heat_input_mmbtu','co2_mass_tons','co2_mass_tons_unadjusted']:
    # identify plants with zero values for both
    plant_ids = list(compared_merged[(compared_merged[f'{col}_calc'] == 0) & (compared_merged[f'{col}_egrid'] == 0)].index)
    compared.loc[compared.index.isin(plant_ids), col] = 1

# for each column, categorize the data based on how far it is off from egrid
for col in ['net_generation_mwh','heat_input_mmbtu','co2_mass_tons','co2_mass_tons_unadjusted']:
    # add a new column
    compared[f'{col}_status'] = pd.cut(x=compared[col], 
                                       bins=[-999999999,0,0.5,0.9,0.99,0.9999,1,1.0001,1.01,1.1,1.5,999999999], 
                                       labels=['negative','<50%','+/-50%','+/-10%','+/-1%','!exact','!exact','+/-1%','+/-10%','+/-50%','>50%'], 
                                       ordered=False)
    # replace any missing values with missing
    compared[f'{col}_status'] = compared[f'{col}_status'].astype(str)  
    compared[f'{col}_status'] = compared[f'{col}_status'].fillna('missing')
    compared[f'{col}_status'] = compared[f'{col}_status'].replace('nan','missing')
    compared.loc[(compared.index.isin(plants_not_in_egrid)),f'{col}_status'] = 'not_in_egrid'

# identify which plants are missing from egrid vs calculated values
for col in ['net_generation_mwh','heat_input_mmbtu','co2_mass_tons','co2_mass_tons_unadjusted']:
    # identify plants that are missing in egrid
    plants_missing_egrid = list(compared_merged[(compared_merged[f'{col}_calc'] > 0) & (compared_merged[f'{col}_egrid'].isna())].index)
    compared.loc[compared.index.isin(plants_missing_egrid), f'{col}_status'] = 'missing_in_egrid'
    # identify plants that are missing from our calculations
    plants_missing_calc = list(compared_merged[(compared_merged[f'{col}_calc'].isna()) & (compared_merged[f'{col}_egrid'] > 0)].index)
    compared.loc[compared.index.isin(plants_missing_calc), f'{col}_status'] = 'missing_in_calc'
    # identify where our calculations are missing a zero value
    plants_missing_zero_calc = list(compared_merged[(compared_merged[f'{col}_calc'].isna()) & (compared_merged[f'{col}_egrid'] == 0)].index)
    compared.loc[compared.index.isin(plants_missing_zero_calc), f'{col}_status'] = 'missing_zero_in_calc'
    # identify where egrid has a missing value instead of a zero
    plants_missing_zero_egrid = list(compared_merged[(compared_merged[f'{col}_calc'] == 0) & (compared_merged[f'{col}_egrid'].isna())].index)
    compared.loc[compared.index.isin(plants_missing_zero_egrid), f'{col}_status'] = 'missing_zero_in_egrid'
    # identify where egrid has a zero value where we have a positive value
    plants_incorrect_zero_egrid = list(compared_merged[(compared_merged[f'{col}_calc'] > 0) & (compared_merged[f'{col}_egrid'] == 0)].index)
    compared.loc[compared.index.isin(plants_incorrect_zero_egrid), f'{col}_status'] = '>50%'

# create a dataframe that counts how many plants are in each category
comparison_count = []
for col in ['net_generation_mwh','heat_input_mmbtu','co2_mass_tons','co2_mass_tons_unadjusted']:
    count = compared.groupby(f'{col}_status', dropna=False).count()['plant_name'].rename(col)
    count.index = count.index.rename('status')
    comparison_count.append(count)

comparison_count = pd.concat(comparison_count, axis=1)
comparison_count = pd.concat([comparison_count, pd.DataFrame(comparison_count.sum().rename('Total')).T], axis=0)   
comparison_count

,net_generation_mwh,heat_input_mmbtu,co2_mass_tons,co2_mass_tons_unadjusted
!exact,10159,8102,7555,8388
+/-1%,27,543,594,728
+/-10%,63,321,402,476
+/-50%,42,280,327,309
<50%,15,10,8,16
>50%,49,772,1180,148
missing,1,10,1,1
missing_in_calc,46,37,23,32
missing_in_egrid,15,61,174,174
missing_zero_in_calc,9,9,32,23


In [83]:
print(comparison_count.to_markdown())

|                       |   net_generation_mwh |   heat_input_mmbtu |   co2_mass_tons |   co2_mass_tons_unadjusted |
|:----------------------|---------------------:|-------------------:|----------------:|---------------------------:|
| !exact                |                10159 |               8102 |            7555 |                       8388 |
| +/-1%                 |                   27 |                543 |             594 |                        728 |
| +/-10%                |                   63 |                321 |             402 |                        476 |
| +/-50%                |                   42 |                280 |             327 |                        309 |
| <50%                  |                   15 |                 10 |               8 |                         16 |
| >50%                  |                   49 |                772 |            1180 |                        148 |
| missing               |                    1 |                

In [ ]:
# examine specific plants in a category
value = 'co2_mass_tons'
status = 'missing'

#compared_merged.loc[64877,:]

compared[compared[f'{value}_status'] == status].sort_values(by=value)

In [ ]:
# TODO: Compare the fuel input from CEMS to the input from EIA to see if they are close
# in general, we will trust the CEMS data over the EIA data unless there are significant differences
# may need to aggregate to plant level since there is not a 1:1 match between units and generators

# for plants where there is data reported in cems, see how off it is from data reported in eia
cems_plant_monthly = cems.groupby(['plant_id_eia','report_date']).sum()[['heat_content_mmbtu']].reset_index()
gf_plant_monthly = gen_fuel_allocated.groupby(['plant_id_eia','report_date']).sum().reset_index()
gf_plant_monthly = gf_plant_monthly.merge(cems_plant_monthly, how='left', on=['plant_id_eia','report_date'])
gf_plant_monthly = gf_plant_monthly[gf_plant_monthly['heat_content_mmbtu'].notnull()]
gf_plant_monthly['pct_diff'] = (gf_plant_monthly['heat_content_mmbtu'] - gf_plant_monthly['fuel_consumed_mmbtu']) / gf_plant_monthly['fuel_consumed_mmbtu']

# identify where there are plants that report 0 heat input to cems but have data in eia_923
gf_plant_monthly[(gf_plant_monthly['heat_content_mmbtu'] == 0) & (gf_plant_monthly['fuel_consumed_mmbtu'] > 0)]

## Compare data at BA level

In [84]:
# load egrid BA totals
egrid_ba = pd.read_excel(f'../data/egrid/egrid{year}_data.xlsx', sheet_name=f'BA{str(year)[-2:]}', header=1, usecols=['BANAME','BACODE','BAHTIANT','BANGENAN','BACO2AN'])
# rename the columns
egrid_ba = egrid_ba.rename(columns={'BANAME':'ba_name',
                                    'BACODE':'ba_code',
                                    'BAHTIANT':'heat_input_mmbtu',
                                    'BANGENAN':'net_generation_mwh',
                                    'BACO2AN':'co2_mass_tons'})

# divide our calculation by the BA totals from eGRID
# if there are 0 values, replace with 0.1, so that div by zero doesn't return missing value
ba_metric = plant_annual_total.groupby('ba_code', dropna=False).sum().drop(columns='plant_id_eia').replace(0,0.1).div(egrid_ba.set_index('ba_code').drop(columns='ba_name').replace(0,0.1)).sort_values(by='co2_mass_tons').round(3)

# calculate the difference in the number of plants in each region
plant_count = (plant_annual_total.groupby('ba_code', dropna=False).count()['plant_id_eia'] - egrid_plant.groupby('ba_code', dropna=False).count()['plant_id_eia']).rename('num_plants')
ba_metric = ba_metric.merge(plant_count, how='left', left_index=True, right_index=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(ba_metric)

,co2_mass_tons,heat_input_mmbtu,net_generation_mwh,num_plants
ba_code,,,,
NaN,0.309,0.363,0.322,-32.0
TEPC,0.859,0.818,0.697,0.0
DEAA,0.997,0.997,1.000,0.0
GRIF,0.999,0.999,1.000,0.0
OVEC,1.000,1.000,1.000,0.0
CSTO,1.000,1.000,1.000,0.0
NSB,1.000,1.000,1.000,0.0
SPA,1.000,1.000,1.000,0.0
CEA,1.000,1.000,1.000,0.0


In [85]:
print(ba_metric.to_markdown())

| ba_code   |   co2_mass_tons |   heat_input_mmbtu |   net_generation_mwh |   num_plants |
|:----------|----------------:|-------------------:|---------------------:|-------------:|
| nan       |           0.309 |              0.363 |                0.322 |          -32 |
| TEPC      |           0.859 |              0.818 |                0.697 |            0 |
| DEAA      |           0.997 |              0.997 |                1     |            0 |
| GRIF      |           0.999 |              0.999 |                1     |            0 |
| OVEC      |           1     |              1     |                1     |            0 |
| CSTO      |           1     |              1     |                1     |            0 |
| NSB       |           1     |              1     |                1     |            0 |
| SPA       |           1     |              1     |                1     |            0 |
| CEA       |           1     |              1     |                1     |            0 |

# Assign monthly data to hourly profile
We now, in theory, have complete data on national-level heat input, net generation, and emissions, from a combination of two sources:
    1. hourly data from CEMS
    2. momthly data for generators that don't report to CEMS

For the second category of monthly data, we need to figure out how to allocate the monthly level data to each hour. 

In [ ]:
# categorize generators by broad fuel categories (clean, geothermal, biofuel, fossil)
clean_fuels = ['SUN','MWH','WND', 'WAT','WH','PUR','NUC']
fossil_fuels = ['NG', 'DFO','OG','WDS','BIT','OTH','PC','SUB', 'LIG','KER', 'RC', 'WO','RFO', 'WC', 'SGC', 'SGP', 'PG', 'JF','BFG']
bio_fuels = ['AB','BG','BLQ','DG','LFG','MSB','MSW','OBG','OBL','OBS','SLW','WDL','WDS']
geo_fuel = ['GEO']
# TODO: Figure out what to do with MSW

# Output data 

Save data to a CSV so we can separate generation of and analysis of hourly e-grid numbers